<a href="https://colab.research.google.com/github/A-sy-coding/Natural-language-processing/blob/main/article_kakao_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링 함수 만들기

In [ ]:
# 필요한 라이브러리 

import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

In [ ]:
# beautifulsoup 객체 생성
# header 매개변수 변경 --> requests의 get 메소드에는 headers 매개변수가 존재

#=> http://www.useragentstring.com에서 지금 접속하는 브라우저의 header정보를 복사하여 python으로 가져옴

def get_soup_obj(url):

  headers = {'User-Agent' : 'Mozilla/5.0'}
  res = requests.get(url, headers = headers)
  soup = BeautifulSoup(res.text,'html.parser')  # html소스 가져오기
                                                 # 첫인자는 html소스, 두번째 인자는 어떤 parser를 이용할지 명시
  return soup                                                 

In [ ]:
# 뉴스의 기본 정보 가져오기

def get_top3_news_info(sec):

  # 상위 뉴스 목록 주소
  sec_url = 'https://news.naver.com/main/home.naver'

  # 상위 뉴스 HTML 가져오기
  soup = get_soup_obj(sec_url)

  # 상위 뉴스 3개 가져오기
  news_list = []

      # div 태그에서 class, id속성값에 해당되는 값들을 구하고, 그 중에서 li 태그를 가진 것들만 고른다.
  news = soup.find("div", attrs={'class':'main_component droppable', 'id':'section_'+sec}).find_all('li', limit=3)
  
  for li in news:
    # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
    news_info = {
            "title" : li.find('strong').text if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "news_url" : li.a.attrs.get('href')}
    news_list.append(news_info)
    
  return news_list

In [ ]:
# 뉴스 이미지/본문 가져오기

def get_news_contents(url):

  soup = get_soup_obj(url)  # html 가져오기
  body = soup.select("div._article_body_contents")[0]

  # 임시 이미지 ( 기사에 사지이 없을 경우 )
  default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
  try:
    news_image = soup.find('span', attrs={'class':'end_photo_org'}).img.attrs.get('src')
  except:
    news_image = default_img

  news_contents = ''
  for content in body:
      if type(content) is bs4.element.NavigableString and len(content) > 50:
          news_contents += content.strip() + ' '    # 뉴스 요약을 위하여 '.' 마침표 뒤에 한칸을 띄워 문장을 구분하도록 함
    
  return news_image, news_contents

In [ ]:
# 정치, 경제, 사회 분야의 상위 3개 뉴스 크롤링

def get_naver_news_top3():

  news_dic = dict()

  # selections -> 정치, 경제, 사회
  sec = ['pol','eco','soc']
  sections = ["politics", "economy","society"]

  for sec, sections in zip(sec, sections):

    # 뉴스의 기본 정보 가져오기
    news_info = get_top3_news_info(sections)

    for news in news_info:

      # 뉴스 본문 가져오기
      news_url = news['news_url']  # 딕셔너리 형태로 저장되어 있는 news_url 가져오기
      news_image, news_contents = get_news_contents(news_url)
      
      # 뉴스 정보를 저장하는 dictionary를 구성
      news['image_url'] = news_image
      news['news_contents'] = news_contents

    news_dic[sec] = news_info
    
  return news_dic

In [ ]:
# '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
news_dic['pol'],news_dic['eco'],news_dic['soc']

([{'image_url': 'https://imgnews.pstatic.net/image/016/2021/09/05/20210905000163_0_20210905155004117.jpg?type=w647',
   'news_contents': '[헤럴드경제=정윤희 기자]이준석 국민의힘 대표는 5일 “정홍원 당 선거관리위원장이 많은 고생을 하고 계시기 때문에 더 큰 성원과 지지, 신뢰를 보낸다는 말씀을 드린다”고 정 위원장에 힘을 실었다. 경선룰을 둘러싼 갈등이 첨예해지며 일부 대선주자들이 당 공식행사에 불참하고, 정 위원장의 사의설까지 나온 것을 염두에 둔 것으로 풀이된다. 이 대표는 이날 서울 여의도 중앙당사에서 열린 대선주자 간담회 및 경선후보 공정경선 서약식에서 홍준표 의원과 유승민 전 의원 등을 겨냥해 “오늘 우리 경선에 서막을 알리는 공정선거 서약 자리에 빠진 자리들이 있는 거 같아서 당 대표로서 매우 유감”이라며 이같이 말했다. 이날 홍 의원과 유 전 의원, 하태경 의원, 안상수 전 인천시장 등 대선주자 4명은 앞서 예고한대로 공정경선 서약식에 불참했다. 이들은 역선택 방지 조항을 도입 않기로 한 경선준비위원회 안을 확정하라고 요구하고 있다. 박찬주 예비역 대장은 당초 ‘보이콧 입장문’에 이름을 올렸지만, 이날 행사에 참석했다. 이 대표는 이들을 겨냥해 “당 선거관리에 전권을 부여받은 선관위의 운영에 다소간의 불만이 있다고 해서 당 공식행사에 불참하는 행위에 대해서 매우 우려스럽고 다시 반복돼선 안 된다는 생각하게 된다”며 “앞으로 주자들 경우에 다소간 이견 있다하더라도 성숙한 방식으로 본인들의 의사를 표현하고 최소한 선관위에 대한 기본적 예의를 지켜야 된다”고 했다. 그러면서 “당 대표로서 말하지만 지난 2012년 총선을 승리로 이끄셨던 공관위원장이셨던, 우리 정부에서 존경받는 총리 역임하신 정홍원 선관위원장께선 지도부에 무한한 신임과 지지를 받고 계신다”며 “항상 우리 당에 어려울 때마다 많은 도움 주시는 정 총리께 당 대표로서 감사하다는 말씀

###################################

# 위의 코드 테스트 해보기

In [ ]:
# 크롤링 테스트
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=014&aid=0004699768'
headers = {'User-Agent' : 'Mozilla/5.0'}
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text,'html.parser')
body = soup.select("div._article_body_contents")[0]
body

<div class="_article_body_contents" id="articleBodyContents">
<!-- 본문 내용 -->
<!-- TV플레이어 -->
<!-- // TV플레이어 -->
<script type="text/javascript">
// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}
</script>
<strong class="media_end_summary">박수현 수석 '브리핑에 없는 대통령 이야기' <br/>"CVC 규제 완화, 대기업-벤처 동반성장 촉진"</strong> <span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/014/2021/08/29/0004699768_001_20210829234204160.jpg?type=w647"><em class="img_desc">[서울=뉴시스] 김진아 기자 = 문재인 대통령이 26일 청와대에서 열린 제2벤처붐 성과보고회 ‘K+벤처’에서 발언을 하고 있다. 2021.08.26. bluesoda@newsis.com /사진=뉴시스</em></img></span> <br/><br/>[파이낸셜뉴스] 청와대는 29일 문재인 정부의 기업형 벤처캐피탈(CVC) 관련 규제 완화 정책이 향후 대기업과 벤처기업의 동반성장을 촉진할 것이라고 기대했다.  <br/> <br/>박수현 청와대 국민소통수석은 이날 자신의 페이스북에 올린 '브리핑에 없는 대통령 이야기-제2 벤처붐을 일으킨 정부의 5대 지원정책'이라는 제목의 글을 통해 이같이 밝혔다.  <br/> <br/>박 수석은 "문재인 정부는 금산분리 원칙 완화에 따른 부작용은 최소화하되, 벤처 활성화를 유도하기 위해 일반지주회사의 CVC 제한적 보유 허용을 추진했다"고 말했다. 이 정책은 총수 일가가 지분을 가진 기업에 투자하지 않는 것 등을 전제로 대기업 지주회사의 CVC 완전자회사 보유를 허용한 것이다. 202

In [ ]:
# 크롤링 테스트
url = 'https://news.naver.com/main/home.naver'
headers = {'User-Agent' : 'Mozilla/5.0'}
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text,'html.parser')
# body = soup.select("div._article_body_contents")[0]
sec = 'politics'
s = soup.find("div", attrs={'class':'main_component droppable', 'id':'section_'+sec}).find_all('li', limit=3)
s[0].a.attrs.get('href') # url 구하기

'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=016&aid=0001883465'

###################################

# 학습한 모델 가져와서 요약하기

In [ ]:
!pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 169.1 MB 28 kB/s 
     |████████████████████████████████| 13.3 MB 136 kB/s 
     |████████████████████████████████| 1.9 MB 7.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1+cpu which is incompatible.


In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

     |████████████████████████████████| 60 kB 4.0 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191620 sha256=d19ca9aefe872314449a47cd930505d35b08bfdaff54f9519733787c9817bc9e
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
     \ 202 kB 337 kB/s
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 
Cloning into 'pyrouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (109/109), done.
2021-09-05 06:49:23,709 [MainThread  ] [INFO ]  Set ROUGE ho

In [ ]:
!pip install transformers
!pip install tensorboardX
!pip install easydict

     |████████████████████████████████| 2.8 MB 6.6 MB/s 
     |████████████████████████████████| 636 kB 48.1 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 124 kB 8.3 MB/s 


In [ ]:
!git clone https://github.com/HaloKim/KorBertSum.git
%cd /content/KorBertSum

Cloning into 'KorBertSum'...
remote: Enumerating objects: 11142, done.
remote: Counting objects: 100% (10840/10840), done.
remote: Compressing objects: 100% (7610/7610), done.
remote: Total 11142 (delta 3259), reused 10780 (delta 3220), pack-reused 302
Receiving objects: 100% (11142/11142), 18.97 MiB | 14.24 MiB/s, done.
Resolving deltas: 100% (3434/3434), done.
/content/KorBertSum


In [ ]:
import os

os.chdir('/content/KorBertSum/src')

# import sys
# sys.path.append("..")

In [ ]:
import torch
import numpy as np
from others.logging import logger, init_logger
from transformers import BertConfig, BertTokenizer
from tensorboardX import SummaryWriter
import easydict

In [ ]:
%cd '/content/KorBertSum/'

import os

os.chdir('/content/KorBertSum/src')

from models import data_loader, model_builder
from models.model_builder import Summarizer
from models.data_loader import load_dataset
from models.reporter import ReportMgr
from models.stats import Statistics

/content/KorBertSum


In [ ]:
def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        logger.info('* number of parameters: %d' % n_params)

    return trainer


class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summ(self, test_iter, step, cal_lead=False, cal_oracle=False):
      """ Validate model.
          valid_iter: validate data iterator
      Returns:
          :obj:`nmt.Statistics`: validation loss statistics
      """
      # Set model in validating mode.
      def _get_ngrams(n, text):
          ngram_set = set()
          text_length = len(text)
          max_index_ngram_start = text_length - n
          for i in range(max_index_ngram_start + 1):
              ngram_set.add(tuple(text[i:i + n]))
          return ngram_set

      def _block_tri(c, p):
          tri_c = _get_ngrams(3, c.split())
          for s in p:
              tri_s = _get_ngrams(3, s.split())
              if len(tri_c.intersection(tri_s))>0:
                  return True
          return False

      if (not cal_lead and not cal_oracle):
          self.model.eval()
      stats = Statistics()

      with torch.no_grad():
          for batch in test_iter:
              src = batch.src
              labels = batch.labels
              segs = batch.segs
              clss = batch.clss
              mask = batch.mask
              mask_cls = batch.mask_cls

              if (cal_lead):
                  selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
              elif (cal_oracle):
                  selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                  range(batch.batch_size)]
              else:
                  sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)
                  sent_scores = sent_scores + mask.float()
                  sent_scores = sent_scores.cpu().data.numpy()
                  selected_ids = np.argsort(-sent_scores, 1)
      return selected_ids



    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

class BertData():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]
        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:2000] for i in idxs]
        src = src[:1000]

        if (len(src) < 3):
            return None

        src_txt = [' '.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        src_txt = [original_src_txt[i] for i in idxs]
        tgt_txt = None
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

def _lazy_dataset_loader(pt_file):
  yield  pt_file

In [ ]:
args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'test',
    "bert_data_path":'/content/drive/MyDrive/BERT_ex/bert_data/korean',
    "model_path":'/content/drive/MyDrive/BERT_ex/bert_data/bert_classifier',
    "result_path":'./results',
    "temp_dir":'./temp',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'/content/drive/MyDrive/BERT_ex/bert_data/logs/log.log',
    "test_from":'/content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt'
})
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

In [ ]:
def test(args, input_list, device_id, pt, step):
  init_logger(args.log_file)
  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  device_id = 0 if device == "cuda" else -1

  cp = args.test_from
  try:
    step = int(cp.split('.')[-2].split('_')[-1])
  except:
    step = 0

  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  if (pt != ''):
      test_from = pt
  else:
      test_from = args.test_from
  logger.info('Loading checkpoint from %s' % test_from)
  checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
  opt = vars(checkpoint['opt'])
  for k in opt.keys():
      if (k in model_flags):
        setattr(args, k, opt[k])

  config = BertConfig.from_pretrained('bert-base-multilingual-cased')
  model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
  model.load_cp(checkpoint)
  model.eval()

  test_iter = data_loader.Dataloader(args, _lazy_dataset_loader(input_list),
                                args.batch_size, device,
                                shuffle=False, is_test=True)
  trainer = build_trainer(args, device_id, model, None)
  result = trainer.summ(test_iter,step)
  return result, input_list

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [ ]:
def txt2input(text):
  data = list(filter(None, text.split('.')))
  bertdata = BertData()
  txt_data = bertdata.preprocess(data)
  data_dict = {"src":txt_data[0],
               "labels":[0,1,2],
               "segs":txt_data[2],
               "clss":txt_data[3],
               "src_txt":txt_data[4],
               "tgt_txt":None}
  input_data = []
  input_data.append(data_dict)
  return input_data

In [ ]:
# input_data = txt2input(text)
# sum_list = test(args, input_data, -1, '', None)
# sum_list[0]
# [list(filter(None, text.split('\n')))[i] for i in sum_list[0][0][:3]]

In [ ]:
# input_data = txt2input(news_info['news_contents'])
# sum_list = test(args, input_data, -1, '', None)
# sum_list

In [ ]:
news_info['news_contents']

'[헤럴드경제=정윤희 기자]이준석 국민의힘 대표는 5일 “정홍원 당 선거관리위원장이 많은 고생을 하고 계시기 때문에 더 큰 성원과 지지, 신뢰를 보낸다는 말씀을 드린다”고 정 위원장에 힘을 실었다. 경선룰을 둘러싼 갈등이 첨예해지며 일부 대선주자들이 당 공식행사에 불참하고, 정 위원장의 사의설까지 나온 것을 염두에 둔 것으로 풀이된다. 이 대표는 이날 서울 여의도 중앙당사에서 열린 대선주자 간담회 및 경선후보 공정경선 서약식에서 홍준표 의원과 유승민 전 의원 등을 겨냥해 “오늘 우리 경선에 서막을 알리는 공정선거 서약 자리에 빠진 자리들이 있는 거 같아서 당 대표로서 매우 유감”이라며 이같이 말했다. 이날 홍 의원과 유 전 의원, 하태경 의원, 안상수 전 인천시장 등 대선주자 4명은 앞서 예고한대로 공정경선 서약식에 불참했다. 이들은 역선택 방지 조항을 도입 않기로 한 경선준비위원회 안을 확정하라고 요구하고 있다. 박찬주 예비역 대장은 당초 ‘보이콧 입장문’에 이름을 올렸지만, 이날 행사에 참석했다. 이 대표는 이들을 겨냥해 “당 선거관리에 전권을 부여받은 선관위의 운영에 다소간의 불만이 있다고 해서 당 공식행사에 불참하는 행위에 대해서 매우 우려스럽고 다시 반복돼선 안 된다는 생각하게 된다”며 “앞으로 주자들 경우에 다소간 이견 있다하더라도 성숙한 방식으로 본인들의 의사를 표현하고 최소한 선관위에 대한 기본적 예의를 지켜야 된다”고 했다. 그러면서 “당 대표로서 말하지만 지난 2012년 총선을 승리로 이끄셨던 공관위원장이셨던, 우리 정부에서 존경받는 총리 역임하신 정홍원 선관위원장께선 지도부에 무한한 신임과 지지를 받고 계신다”며 “항상 우리 당에 어려울 때마다 많은 도움 주시는 정 총리께 당 대표로서 감사하다는 말씀을 드린다”고 힘을 실었다. '

In [ ]:
##########
 # 뉴스 요약하기

sections = ['pol','eco','soc']

for section in sections:

  for news_info in news_dic[section]:
    input_data = txt2input(news_info['news_contents'])  # 전처리
    sum_list = test(args, input_data, -1, '', None)
    # get_index = sum_list[0]
    news_info['news_contents_summ'] = [list(filter(None, news_info['news_contents'].split('.')))[i] for i in sum_list[0][0][:3]]

  # 요약 결과
  print('\n[', section,']')
  for i in range(3):
    print('\nOriginal)')
    print(news_dic[section][i]['news_contents'])
    print('\nSummary)')
    print(news_dic[section][i]['news_contents_summ'])
    print('\n')

# news_info['news_contents']에서 네이버 뉴스를 가져온뒤, 학습시킨 모델에 넣고 요약문을 얻은 뒤, 
# news_info에서 'news_contents_summ' 변수에 요약문을 새로 지정한다.

[2021-09-05 07:13:05,812 INFO] Loading checkpoint from /content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt
[2021-09-05 07:13:12,423 INFO] * number of parameters: 177854209


gpu_rank 0


[2021-09-05 07:13:18,246 INFO] Loading checkpoint from /content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt
[2021-09-05 07:13:24,844 INFO] * number of parameters: 177854209


gpu_rank 0


[2021-09-05 07:13:30,453 INFO] Loading checkpoint from /content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt
[2021-09-05 07:13:37,016 INFO] * number of parameters: 177854209


...

gpu_rank 0


[2021-09-05 07:14:31,014 INFO] Loading checkpoint from /content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt
[2021-09-05 07:14:37,578 INFO] * number of parameters: 177854209


gpu_rank 0


[2021-09-05 07:14:43,130 INFO] Loading checkpoint from /content/drive/MyDrive/BERT_ex/bert_data/bert_classifier/model_step_6000.pt
[2021-09-05 07:14:49,672 INFO] * number of parameters: 177854209


gpu_rank 0

[ soc ]

Original)
(포항=연합뉴스) 손대성 기자 = 장기간 공사 중단으로 흉물로 방치된 경북 포항 아파트단지가 철거에 들어갔다. 5일 포항시에 따르면 북구 용흥동 금광포란재 아파트 부지 소유권자인 솔빛주택건설 등은 지난 3일 철거를 시작했다. 철거 공사 착공식에는 이강덕 시장과 김정재 국회의원을 비롯해 도의원과 시의원 등이 참석했다. 금광포란재 아파트는 지난 1997년 지하 4층, 지상 15층, 314가구 규모로 주택건설 사업계획 승인이 났다. 이후 여러 차례 사업자 변경을 거쳐 공정률 40% 상태에서 사업 주체가 부도나면서 공사가 중단됐다. 이런 상황에서 토지가 제3자에게 경매 처분돼 권리관계 분쟁이 발생했고 수년간 소송을 거치면서 도심 속 흉물로 방치되어 왔다. 시는 최근 법원 판결을 바탕으로 지난 5월 3일 토지소유자인 솔빛주택 신청에 따라 사업승인을 취소했다. 이강덕 시장은 "시의 적극적인 노력으로 숙원사업이 해결돼 기쁘고 새로운 명품 아파트 건설로 용흥동이 발전하는 계기가 되기를 바란다"고 말했다. 

Summary)
[' 5일 포항시에 따르면 북구 용흥동 금광포란재 아파트 부지 소유권자인 솔빛주택건설 등은 지난 3일 철거를 시작했다', ' 금광포란재 아파트는 지난 1997년 지하 4층, 지상 15층, 314가구 규모로 주택건설 사업계획 승인이 났다', ' 이런 상황에서 토지가 제3자에게 경매 처분돼 권리관계 분쟁이 발생했고 수년간 소송을 거치면서 도심 속 흉물로 방치되어 왔다']



Original)
소득 하위 88%가 1인당 25만 원씩 받는 '코로나 상생 국민지원금(5차 재난지원금)' 접수가 6일부터 시작된다. 가구별로 세대주가 신청했던 지난해와 달리  성인은 개인별로 각각 신청할 수 있으며, 신청 첫 주에는 출생연도 끝자리에 따라 요일별로 순차적으로 접수가 진행된다. 행정안전부에 따르면, 6일 오전 9시부터 온라인에서 국민지원금 지급 대상 여부 조회 및 지급 신청이 가능하다.  오프라인은 이달 13일부터 

# 카카오톡으로 연결하기

In [ ]:
# REST API 키	 --> 504d0e40f7bc274e44568959ace57bb3
# code         --> adrGV5gltu1PxQPz4WzAq0bdxM9Cv711DHx-R-Fmd2o0lTGU32rKZRuFiw3ONh8O9mgqygorDKcAAAF7sgaedw

In [ ]:
import requests
import json

In [ ]:
rest_api_key = "504d0e40f7bc274e44568959ace57bb3"
redirect_url = "https://localhost:3000"
code = "koYt5b2USnoiUwQal47wSbHB6HHtW3qJItsV5Ls5bMuk415x_kfjnawjZOzW7oy37XJ6Cgo9cxcAAAF7tPJejQ"

In [ ]:
# 카카오톡 메세지 API
url = 'https://kauth.kakao.com/oauth/token'

data = {
    "grant_type" : "authorization_code",
    "client_id" : rest_api_key,
    "redirect_uri" : redirect_url,
    "code"         : code
}

response = requests.post(url, data=data)
tokens = response.json()
print(tokens)

{'access_token': '1KHJK4cXr9tBpT57gh7ndFfSXny3igsmx_YGlwo9dRoAAAF7tPMy_Q', 'token_type': 'bearer', 'refresh_token': 'r47FrYwGy-Em0tTV1PirKdZx8T4HpRzc1MBWHQo9dRoAAAF7tPMy_Q', 'expires_in': 21599, 'scope': 'talk_message', 'refresh_token_expires_in': 5183999}


In [ ]:
# 토큰을 json파일로 저장
with open("/content/drive/MyDrive/BERT_ex/kakao_connect/kakao_token.json", "w") as fp:
    json.dump(tokens, fp)

# 카카오톡으로 기사 보내기

In [ ]:
# 필요한 라이브러리 
import json
import os
import datetime
import requests

In [ ]:
import json
import os
import datetime
import requests

# 저장하는 함수
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

# 읽어오는 함수
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# refresh_token으로 access_token 갱신하는 함수
def update_tokens(app_key, filename) : 
    tokens = load_tokens(filename)

    url = "https://kauth.kakao.com/oauth/token"
    data = {
        "grant_type" : "refresh_token",
        "client_id"  : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    response = requests.post(url, data=data)

    # 요청에 실패했다면,
    if response.status_code != 200:
        print("error! because ",  response.json())
        tokens = None
    else: # 성공했다면,
        print(response.json())
        # 기존 파일 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename+"."+ now
        os.rename(filename, backup_filename)
        # 갱신된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# 메시지 전송 함수
def send_message(filename, template):
    tokens = load_tokens(filename)
    
    headers = {
        "Authorization": "Bearer " + tokens['access_token']
    }
    
    # JSON 형식 -> 문자열 변환
    payload = {
        "template_object" : json.dumps(template)
    }

    # 카카오톡 보내기
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    res = requests.post(url, data=payload, headers=headers)

    return res

In [ ]:
#token이 저장된 파일
KAKAO_TOKEN_FILENAME = "/content/drive/MyDrive/BERT_ex/kakao_connect/kakao_token.json"
KAKAO_APP_KEY = "504d0e40f7bc274e44568959ace57bb3"
update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)  # 아직은 토큰이 유효하다.

{'access_token': 'q7QgZKZdPphyEgHfQ2ymsSATVuGVy5PQ5sNYGgo9dJkAAAF7tPNo4A', 'token_type': 'bearer', 'expires_in': 21599}


{'access_token': 'q7QgZKZdPphyEgHfQ2ymsSATVuGVy5PQ5sNYGgo9dJkAAAF7tPNo4A',
 'expires_in': 21599,
 'refresh_token': 'r47FrYwGy-Em0tTV1PirKdZx8T4HpRzc1MBWHQo9dRoAAAF7tPMy_Q',
 'refresh_token_expires_in': 5183999,
 'scope': 'talk_message',
 'token_type': 'bearer'}

In [ ]:
# 분야 지정
my_section = "eco"

# 사용자가 선택한 카테고리를 제목에 넣기 위한 dictionary
sections_ko = {'pol': '정치', 'eco' : '경제', 'soc' : '사회'}

# 네이버 뉴스 URL
navernews_url = "https://news.naver.com/main/home.naver"

# 추후 각 리스트에 들어갈 내용(content) 만들기
contents = []

# 리스트 템플릿 형식 만들기
template = {
    "object_type" : "list",
    "header_title" : sections_ko[my_section] + " 분야 상위 뉴스 빅3",
    "header_link" : {
        "web_url": navernews_url,
        "mobile_web_url" : navernews_url
    },
    "contents" : contents,
    "button_title" : "네이버 뉴스 바로가기"
}

## 내용 만들기
# 각 리스트에 들어갈 내용(content) 만들기
for news_info in news_dic[my_section]:  # 위에서 구한 news_dic에서 my_section에서 지정한 분야를 가져온다.
    content = {
        "title" : news_info.get('title'),
        "image_url" : news_info.get('image_url'),
        "image_width" : 50, "image_height" : 50,
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        }
    }

    contents.append(content)

In [ ]:
# 카카오톡 메시지 전송
res = send_message(KAKAO_TOKEN_FILENAME, template)

if res.json().get('result_code') == 0:
    print('뉴스를 성공적으로 보냈습니다.')
else:
    print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

뉴스를 성공적으로 보냈습니다.


In [ ]:
# 각 뉴스의 요약 결과를 전송
for idx, news_info in enumerate(news_dic[my_section]):
    # 텍스트 템플릿 형식 만들기
    template = {
        "object_type": "text",
        "text": '# 제목 : ' + news_info.get('title') + \
                '\n\n# 요약 : ' + news_info.get('news_contents_summ')[0],  # 기사요약 정보가 들어간다.
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        },
        "button_title": "자세히 보기"
    }
    
    # 카카오톡 메시지 전송
    res = send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('뉴스를 성공적으로 보냈습니다.')
    else:
        print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.
